In [1]:
import pandas as pd
import sys
from io import StringIO, BytesIO
import ipywidgets as widgets
import os


pd.set_option('display.max_columns', 3000)
pd.set_option('display.max_rows', 3000)
pd.set_option('display.notebook_repr_html', True)

from io_module import get_sheet_list, get_file_list, split_excel_table, debug_output, save_csv, experimental_split
from reformat import filter_column, filter_row, list_mapper, get_year, get_month
import inspect

import warnings
warnings.filterwarnings('ignore')

import json

In [2]:
#File

up = widgets.FileUpload(accept='.xlsx,.xls', multiple=False)

select_sheet =  widgets.Dropdown(
    options=[''],
    value='',
    description='Select Sheet',
    disabled=False)

button_preview_file = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')

#Region

select_cluster_mode = widgets.Dropdown(
    options=['Option 1', 'Option 2'],
    value='Option 1',
    description='',
    disabled=False)

cluster_threshold = widgets.IntText(
    value=1,
    description='Threshold',
    disabled=False
)

button_preview_region = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')


#Save Config
save_name = widgets.Text(
    value='',
    placeholder='Type something',
    description='',
    disabled=False
)

button_save = widgets.Button(
    description='Save',
    disabled=False,
    button_style='info',
    tooltip='Click to Save Config',
    icon='bar')



#Serve
up_multiple = widgets.FileUpload(
    accept='.xlsx,.xls', 
    multiple=True, 
    layout=widgets.Layout(flex='1 1 auto', width='auto'))

existing_config = widgets.Dropdown(
    options=[''],
    value='',
    description='',
    disabled=False,
    layout=widgets.Layout(flex='1 1 auto', width='auto'))

button_preview_serve = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')



#Reformat
reformat_option = widgets.Dropdown(
    options=['Choose Cluster','Filter Row', 'Filter Column', 'Merge', 
             'Unmerge', 'Transpose', 'Pivot', 'Unpivot', 'Rename Columns'],
    value='Filter Row',
    description='',
    disabled=False)

add_option = widgets.Button(
    description='',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='plus', # (FontAwesome names without the `fa-` prefix)
    layout=widgets.Layout(flex='0.05 1 auto', width='auto')
)

# 3 dropdown option
cluster_option = widgets.Dropdown(
    options=[''],
    value='',
    description='',
    disabled=False)

filter_row_option = widgets.Dropdown(
    options=['retain', 'remove'],
    value='retain',
    description='',
    disabled=False)

filter_column_option = widgets.Dropdown(
    options=['retain', 'remove'],
    value='retain',
    description='',
    disabled=False)

#5 String TExt
filter_row_text = widgets.Text(
    value='',
    placeholder='Type Unique Marker',
    description='',
    disabled=False
)

filter_column_text = widgets.Text(
    value='',
    placeholder='Type Unique Marker',
    description='',
    disabled=False
)

merge_column_text = widgets.Text(
    value='',
    placeholder='Type Index',
    description='',
    disabled=False
)

unpivot_text = widgets.Text(
    value='',
    placeholder='Type id variable',
    description='',
    disabled=False
)

rename_column_text = widgets.Text(
    value='',
    placeholder='Type column names',
    description='',
    disabled=False
)

# 6 preview
cluster_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')

filter_row_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')

filter_column_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')

merge_column_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')

unpivot_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')

rename_column_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')


### ExcelParser

In [3]:
from openpyxl import load_workbook
#tab = widgets.Tab()    
out_file = widgets.Output(layout={'border': '1px solid black'})
out_region = widgets.Output(layout={'border': '1px solid black'})
out_reformat = widgets.Output(layout={'border': '1px solid black'})
out_serve = widgets.Output(layout={'border': '1px solid black'})

layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%')

tab = widgets.Tab(layout = widgets.Layout(max_height='800px', max_width='1200px', overflow='visible'))    

items_file = [
    widgets.Label('Select Files', layout=widgets.Layout(flex='0.1 1 auto', width='auto')),
    up,
    widgets.Label('', layout=widgets.Layout(flex='1 1 auto', width='auto')),
    select_sheet,
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    button_preview_file

 ]

item_region = [
    widgets.Label('Mode', layout=widgets.Layout(flex='0.1 1 auto', width='auto')),
    select_cluster_mode,
    widgets.Label('', layout=widgets.Layout(flex='1 1 auto', width='auto')),
    cluster_threshold,
    widgets.Label('', layout=widgets.Layout(flex='1 1 auto', width='auto')),
    button_preview_region
]


item_save = [
    widgets.Label('Config Name', layout=widgets.Layout(flex='0.05 1 auto', width='auto')),
    save_name,
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    button_save
    
]

item_reformat = [
    #widgets.Label('Pick Option', layout=widgets.Layout(flex='0.1 1 auto', width='auto')),
    reformat_option,
    add_option
]

item_reformat_cluster = [
    widgets.Label('Cluster', layout=widgets.Layout(flex='0.1 1 auto', width='auto')),
    cluster_option,
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    cluster_preview

]

item_reformat_filter_row = [
    widgets.Label('Mode', layout=widgets.Layout(flex='0.1 1 auto', width='auto')),
    filter_row_option,
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    filter_row_text,
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    filter_row_preview
]

item_reformat_filter_column = [
    widgets.Label('Mode', layout=widgets.Layout(flex='0.1 1 auto', width='auto')),
    filter_column_option,
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    filter_column_text,
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    filter_column_preview
]

item_reformat_merge_column = [
    widgets.Label('Index', layout=widgets.Layout(flex='0.1 1 auto', width='auto')),
    merge_column_text,
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    merge_column_preview
]

item_reformat_unpivot = [
    widgets.Label('ID Column', layout=widgets.Layout(flex='0.1 1 auto', width='auto')),
    unpivot_text,
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    unpivot_preview
]

item_reformat_rename_column = [
    widgets.Label('New Columns', layout=widgets.Layout(flex='0.1 1 auto', width='auto')),
    rename_column_text,
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    rename_column_preview
]


item_serve = [
    widgets.Label('Excel File(s)', layout=widgets.Layout(flex='0.1 1 auto', width='auto')),
    up_multiple,
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    widgets.Label('Config File', layout=widgets.Layout(flex='0.1 1 auto', width='auto')),
    existing_config,
    widgets.Label('', layout=widgets.Layout(flex='1.5 1 auto', width='auto')),
    button_preview_serve
    
]

box_layout = widgets.Box(children=items_file, layout = layout)
box_region = widgets.Box(children=item_region, layout = layout)
box_save = widgets.Box(children=item_save, layout = layout)
box_serve = widgets.Box(children=item_serve, layout = layout)
box_reformat = widgets.Box(children = item_reformat, layout = layout)


#REFORMAT

box_cluster = widgets.Box(children = item_reformat_cluster, layout = layout)
box_filter_row = widgets.Box(children = item_reformat_filter_row, layout = layout)
box_filter_column = widgets.Box(children = item_reformat_filter_column, layout = layout)

box_merge_column = widgets.Box(children = item_reformat_merge_column, layout = layout)
box_unpivot = widgets.Box(children = item_reformat_unpivot, layout = layout)
box_rename_column = widgets.Box(children = item_reformat_rename_column, layout = layout)


accordion = widgets.Accordion(children=[
    widgets.VBox([box_cluster]),  widgets.VBox([box_filter_row]), widgets.VBox([box_filter_column]), 
    widgets.VBox([box_merge_column]), widgets.VBox([box_unpivot]), widgets.VBox([box_rename_column]),

])

accordion.set_title(0, 'Choose Cluster')
accordion.set_title(1, 'Filter Row')
accordion.set_title(2, 'Filter Column')
accordion.set_title(3, 'Merge')
# accordion.set_title(4, 'Unmerge')
# accordion.set_title(5, 'Transpose')
# accordion.set_title(6, 'Pivot')
accordion.set_title(4, 'Unpivot')
accordion.set_title(5, 'Rename Columns')


children = [
    widgets.VBox([box_layout, out_file]),
    widgets.VBox([box_region, out_region]),
    widgets.VBox([accordion, out_reformat]),
    widgets.VBox([box_save]),
    widgets.VBox([box_serve, out_serve])
]

tab.children = children
tab.set_title(0, "Choose File")
tab.set_title(1, "Auto Split")
tab.set_title(2, "Reformat")
tab.set_title(3, "Save Config")
tab.set_title(4, "Serve & Parse")
display(tab)

excel_file = None


def change_sheet_options():
    ex = get_file_content()
    select_sheet.options = ex.sheet_names

def get_bytes_content():
    if up.value != None:
        raw_file = up.value
        
        for key in raw_file:
            fname = key
        content = raw_file[fname]['content']
        
        bio = BytesIO(content)
        return bio
    
def get_file_content():
    bio = get_bytes_content()
    excel_data = pd.ExcelFile(bio)
    return excel_data
    
def df_converter():
    sheet = select_sheet.value
    ex = get_file_content()
    df = ex.parse(sheet, header=None)
    return df
    
    
def df_split():
    mode = select_cluster_mode.value
    thres = cluster_threshold.value
    sheet = select_sheet.value
    bio = get_bytes_content()

    df_list = []
    if(mode=='Option 1'):
        df_list = split_excel_table(bio,sheet,thres=thres,debug=False)
        df_list = sorted(df_list,key=len, reverse=True)
    else:
        print('In Development')
    
    return df_list


def df_cluster():
    df_list = df_split()
    selected = cluster_option.value
    df = df_list[selected]
    return df


def df_filter_row():
    df = df_cluster()
    mode = filter_row_option.value
    
    marker = filter_row_text.value
    list_marker = marker.split(',')
    list_marker = [a.split()[0] for a in list_marker] 
    
    df = filter_row(df, list_marker, mode=mode)
    return df
    
    
def df_filter_column():
    df = df_filter_row()
    mode = filter_column_option.value

    marker = filter_column_text.value
    list_marker = marker.split(',')
    list_marker = [' '.join(a.split()) for a in list_marker] 
    
    df = filter_column(df, list_marker, mode=mode)
    return df


def df_merge_column():
    df = df_filter_column()
    
    marker = merge_column_text.value
    list_marker = marker.split(',')
    list_marker = [int(a.split()[0]) for a in list_marker] 
    
    df.iloc[:,0].fillna('', inplace=True)
    arr = []
    for idx in list_marker:
        df.loc[idx,:] = df.loc[idx,:].fillna(method='ffill')
        arr.append(df.loc[idx,:].values)
        
    df.columns = pd.MultiIndex.from_arrays(arr)
    df.drop(list_marker, axis=0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

    
def df_unpivot():
    df = df_merge_column()
    marker = unpivot_text.value
    
    df = pd.melt(df,id_vars=marker)
    return df

def df_rename_column():
    df = df_unpivot()
    
    marker = rename_column_text.value
    list_marker = marker.split(',')
    list_marker = [' '.join(a.split()) for a in list_marker] 
 
    df.columns = list_marker
    return df


def str_to_list(marker, typ=str):
    
    list_marker = marker.split(',')

    if typ == str:
        list_marker = [' '.join(a.split()) for a in list_marker] 
    else:
        list_marker = [int(a.split()[0]) for a in list_marker]
    return list_marker
    

def save_config():
    
    # auto cluster
    cluster_mode = select_cluster_mode.value
    cluster_thres = cluster_threshold.value
    
    # region
    region = cluster_option.value
    
    # filter row
    fr_mode = filter_row_option.value
    fr_marker = filter_row_text.value
    fr_marker = str_to_list(fr_marker)
    
    # filter column
    fc_mode = filter_column_option.value
    fc_marker = filter_column_text.value
    fc_marker = str_to_list(fc_marker)

    # merge
    merge_marker = merge_column_text.value
    merge_marker = str_to_list(merge_marker, int)
    
    # unpivot
    unpivot_marker = unpivot_text.value
    unpivot_marker = str_to_list(unpivot_marker)
    
    
    # rename
    rename_marker = rename_column_text.value
    rename_marker = str_to_list(rename_marker)
    
    
    json_conf = {
        'auto_split' : {'mode':cluster_mode, 'threshold':cluster_thres},
        'cluster': region,
        'filter_row' : {'mode':fr_mode, 'marker':fr_marker},
        'filter_column' : {'mode':fc_mode, 'marker':fc_marker},
        'merge' : merge_marker,
        'unpivot':unpivot_marker,
        'rename' : rename_marker
    }
    
    # Write to local 
    fname = save_name.value + '.json'

    with open(fname, 'w') as fp:
        json_dumps_str = json.dumps(json_conf, indent=4)
        print(json_dumps_str, file=fp)

    
def get_existing_config():
    config_list = []
    
    for file in os.listdir(os.getcwd()):
        if file.endswith(".json"):
            config_list.append(file.split('/')[-1])
    return config_list


def get_full_path(fname):
    return os.path.abspath(fname)    
    
    
def parse_excel_with_json(bio, sheet, config_name):
    
    with open(config_name) as json_file:
        config = json.load(json_file)
    
    #Clustering
    auto_split_option = config['auto_split']['mode']
    auto_split_thres = config['auto_split']['threshold']
    cluster_region = config['cluster']
    
    df_list = None
    if auto_split_option == 'Option 1':
        df_list = split_excel_table(bio,sheet,thres=auto_split_thres,debug=False)
        
    df_list = sorted(df_list,key=len, reverse=True)
    df = df_list[cluster_region]
    
    #Filter Row
    filter_row_list = config['filter_row']['marker']
    filter_row_mode = config['filter_row']['mode']
    df = filter_row(df, filter_row_list, mode=filter_row_mode)
    
    #Filter Column
    filter_column_list = config['filter_column']['marker']
    filter_column_mode = config['filter_column']['mode']
    df = filter_column(df, filter_column_list, mode=filter_column_mode)

    #Merge        
    index_merge = config['merge']
    df.iloc[:,0].fillna('', inplace=True)
    
    arr = []
    for idx in index_merge:
        df.loc[idx,:] = df.loc[idx,:].fillna(method='ffill')
        arr.append(df.loc[idx,:].values)       
    df.columns = pd.MultiIndex.from_arrays(arr)
    df.drop(index_merge, axis=0, inplace=True)
    df.reset_index(drop=True, inplace=True)
     
    #Unpivot
    id_col = config['unpivot']
    df = pd.melt(df,id_vars=id_col)       
        
    #Rename
    new_col = config['rename']
    df.columns = new_col
    
    return df
    
def parse_serve_multi_files():
    
    config = existing_config.value
    df_list = []
    meta_list = []
    
    if up_multiple.value != None:
        
        files = up_multiple.value.keys()
        for file in files:
            content = up_multiple.value[file]['content']
            bio = BytesIO(content)
            ex = pd.ExcelFile(bio)
            
            sheet_list = ex.sheet_names
            for sheet in sheet_list:
                df = parse_excel_with_json(bio, sheet, config)
                df_list.append(df)
                meta_list.append('File: {}, Sheet: {}'.format(file, sheet))

            
    return df_list, meta_list
    
    
    
# PREVIEW    
def preview():
    df = df_converter()
    out_file.clear_output()
    with out_file:
        display(df)
        
def preview_region():
    df_list = df_split()
    
    cluster_option.options = range(len(df_list))
    out_region.clear_output()
    with out_region:
        
        for idx, df in enumerate(df_list):
            print('\n Cluster {}'.format(idx))
            display(df)
    
    
def preview_cluster():
    df = df_cluster()
    
    with out_reformat:
        out_reformat.clear_output()
        display(df)

        
def preview_filter_row():
    df = df_filter_row()
    
    with out_reformat:
        out_reformat.clear_output()
        display(df)

        
def preview_filter_column():
    df = df_filter_column()
    
    out_reformat.clear_output()
    with out_reformat:
        display(df)
        
        
def preview_merge_column():
    df = df_merge_column()
    
    out_reformat.clear_output()
    with out_reformat:
        display(df)
        
        
def preview_unpivot():
    df = df_unpivot()
    
    out_reformat.clear_output()
    with out_reformat:
        display(df)

        
def preview_rename_column():
    df = df_rename_column()
    
    out_reformat.clear_output()
    with out_reformat:
        display(df)
        
def preview_serve_parse():
    df_list, meta_list = parse_serve_multi_files()
    
    with out_serve:
        out_serve.clear_output()
        for df, meta in zip(df_list,meta_list):
            print('\n',meta)
            display(df)
    
def on_upload_change(change):
    #clear_output(wait=True)
    change_sheet_options()

def preview_clicked(b):
    preview()        

def preview_region_clicked(b):
    preview_region()    

def preview_cluster_clicked(b):
    preview_cluster()
    
def preview_filter_row_clicked(b):
    preview_filter_row()

def preview_filter_column_clicked(b):
    preview_filter_column()

def preview_merge_column_clicked(b):
    preview_merge_column()

def preview_unpivot_clicked(b):
    preview_unpivot()
    
def preview_rename_column_clicked(b):
    preview_rename_column()

def button_save_config(b):
    save_config()
    existing_config.options = get_existing_config() 

def preview_parse_serve_clicked(b):
    preview_serve_parse()
    
    
existing_config.options = get_existing_config() 
# Event Handler    

up.observe(on_upload_change, names='_counter')
button_preview_file.on_click(preview_clicked)
button_preview_region.on_click(preview_region_clicked)

# Region
cluster_preview.on_click(preview_cluster_clicked)
filter_row_preview.on_click(preview_filter_row_clicked)
filter_column_preview.on_click(preview_filter_column_clicked)
merge_column_preview.on_click(preview_merge_column_clicked)
unpivot_preview.on_click(preview_unpivot_clicked)
rename_column_preview.on_click(preview_rename_column_clicked)

# Save Config
button_save.on_click(button_save_config)

# Parse Serve
button_preview_serve.on_click(preview_parse_serve_clicked)